<a href="https://colab.research.google.com/github/xuanngoczxc/Profile/blob/main/FakeNewTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nhập thư viện

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Thư viện NLP để làm sạch dữ liệu văn bản
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

# Kỹ thuật vector hóa TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Để tách tập dữ liệu
from sklearn.model_selection import train_test_split

# Thư viện mẫu
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

#Thư viện đo độ chính xác
from sklearn.metrics import accuracy_score


Tải dữ liệu lên

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Lê Xuân Ngọc 44A/data.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data.shape

(4009, 4)

In [ ]:
data.head()

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1


In [ ]:
data.columns

Index(['URLs', 'Headline', 'Body', 'Label'], dtype='object')

In [ ]:
data.isnull().sum()

URLs         0
Headline     0
Body        21
Label        0
dtype: int64

Tiền xử lý dữ liệu

In [ ]:
df = data.copy()

Loại bỏ các giá trị NULL

In [ ]:
df['Headline'] = df['Headline'].fillna('')

In [ ]:
df.isnull().sum()

URLs         0
Headline     0
Body        21
Label        0
dtype: int64

Thêm một cột mới

In [ ]:
df['News'] = df['Headline']

In [ ]:
df.head()

,URLs,Headline,Body,Label,News
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1,Four ways Bob Corker skewered Donald Trump
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1,Linklater's war veteran comedy speaks to moder...
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1,Trump’s Fight With Corker Jeopardizes His Legi...
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1,Egypt's Cheiron wins tie-up with Pemex for Mex...
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1,Jason Aldean opens 'SNL' with Vegas tribute


In [ ]:
df.columns

Index(['URLs', 'Headline', 'Body', 'Label', 'News'], dtype='object')

Bỏ đi những tính năng không cần thiết

In [ ]:
features_dropped = ['URLs','Headline','Body']
df = df.drop(features_dropped, axis =1)

In [ ]:
df.columns

Index(['Label', 'News'], dtype='object')

Xử lý văn bản

In [ ]:
ps = PorterStemmer()
def wordopt(text):
    text = re.sub('[^a-zA-Z]', ' ',text)
    text = text.lower()
    text = text.split()
    text = [ps.stem(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    return text

In [ ]:
import nltk
nltk.download('stopwords')
df['News'] = df['News'].apply(wordopt)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df.head()

,Label,News
0,1,four way bob corker skewer donald trump
1,1,linklat war veteran comedi speak modern americ...
2,1,trump fight corker jeopard legisl agenda
3,1,egypt cheiron win tie pemex mexican onshor oil...
4,1,jason aldean open snl vega tribut


Tách tập dữ liệu

In [ ]:
X = df['News']
Y = df['Label']

#Chia dữ liệu thành tập huấn luyện và tập kiểm tra
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

Vector hóa

In [ ]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [ ]:
import joblib

In [ ]:
joblib.dump(vectorization, '/content/drive/MyDrive/Lê Xuân Ngọc 44A/vectorization.pkl')

['/content/drive/MyDrive/Lê Xuân Ngọc 44A/vectorization.pkl']

Lắp mô hình

In [ ]:
svm_model = SVC(kernel='linear')

#Lắp tập huấn luyện vào mô hình
svm_model.fit(xv_train,y_train)

#Dự đoán kết quả của tập kiểm tra dựa trên mô hình
svm_y_pred = svm_model.predict(xv_test)

#Tính điểm chính xác của mô hình này
score = accuracy_score(y_test,svm_y_pred)
print('Accuracy of SVM model is ', score)

Accuracy of SVM model is  0.8604187437686939


Kiểm tra mô hình thủ công

In [ ]:
# Vì SVM có thể cung cấp kết quả tốt nhất - SVM sẽ được sử dụng để kiểm tra trách nhiệm pháp lý về tin tức
def fake_news_det(news):
    input_data = {"text":[news]}
    new_def_test = pd.DataFrame(input_data)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    #print(new_x_test)
    vectorized_input_data = vectorization.transform(new_x_test)
    prediction = svm_model.predict(vectorized_input_data)

    if prediction == 1:
        print("Not a Fake News")
    else:
        print("Fake News")

In [ ]:
fake_news_det("Four ways Bob Corker skewered Donald Trump")

Not a Fake News


In [ ]:
fake_news_det("Trends to Watch")

Fake News


In [ ]:
fake_news_det('Ron Paul on Trump, Anarchism & the AltRight')

Fake News


In [ ]:
import pickle
from sklearn.svm import SVC


In [ ]:
# Huấn luyện mô hình SVM (fake_news_detector)
# ...

# Lưu mô hình thành file pickle
with open('/content/drive/MyDrive/Lê Xuân Ngọc 44A/fake_news_detector.pkl', 'wb') as model_file:
    pickle.dump(svm_model, model_file)
